# Практическое задание к уроку 4. Сверточные нейронные сети

### Практическое задание

#### Вариант 1. (простой)

- обучить сверточную нейронную сеть в стиле AlexNet (с падением размера ядра свертки и последовательностью блоков свертка-пулинг (conv-pool)-(conv-pool)-...) на датасете fashion-mnist или cifar10

- оценить рост точности при увеличении ширины сети (больше ядер)

- оценить рост точности при увеличении глубины сети (больше слоев)

- (опциционально)сравнить с точностью полносвязной сети для этой выборки

In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install tensorflow -latest


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -l


In [3]:
!pip install --upgrade tensorflow

In [4]:
!pip install mnist

In [5]:
!pip install tensorflow_datasets

In [6]:
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers
import tensorflow.keras as keras
from keras.layers import Dense
# from keras.utils import to_categorical

from tensorflow.keras.utils import to_categorical
from keras.datasets import fashion_mnist

#### Обучение сверточной нейронной сети в стиле AlexNet (с падением размера ядра свертки и последовательностью блоков свертка-пулинг (conv-pool)-(conv-pool)-...) на датасете fashion-mnist

In [7]:
(x_train, y_train), (x_test, y_test)=fashion_mnist.load_data()

In [12]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

y_train = to_categorical(y_train, 10) 
y_test = to_categorical(y_test, 10) 

x_train = x_train.reshape(x_train.shape[0], 28,28,1)
x_test = x_test.reshape(x_test.shape[0], 28,28,1)

#### Оценка роста точности при увеличении ширины сети (больше ядер)

In [16]:
results = []

In [17]:
#по 10 прогонов с каждым из трёх вариантов размера ядер ( 20 - 10 - 5, 10 - 5 - 3, 5 - 3 - 2) 
# с каждым из двух вариантов кол-ва нейронов для сверточного слоя (6 - 16 - 120, 12-32-120)

for i in range (11):
  result = {}

  # инициализация пустой модели
  model = Sequential()

  # первый сверточный слой
  model.add(layers.Conv2D(12, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=(28,28,1), padding="same"))

  # второй пуллинговый слой
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))

  # третий сверточный слой
  model.add(layers.Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='valid'))

  # четвертый пуллинговый слой
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

  # пятый сверточный слой
  model.add(layers.Conv2D(120, kernel_size=(2, 2), strides=(1, 1), activation='relu', padding='valid'))

  # сглаживание CNN выхода чтобы можно было его присоединить к полносвязногому слою
  model.add(layers.Flatten())

  # шестой полносвязный слой
  model.add(layers.Dense(84, activation='relu'))

  # выходной слой с функцией активации softmax
  model.add(layers.Dense(10, activation='softmax'))


  # компилияция модели
  model.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])


  hh = model.fit(x=x_train,y=y_train, epochs=2, batch_size=128, validation_data=(x_test, y_test), verbose=1)

  result['neurons'] = '12, 32, 120'
  result['kernels'] = '5, 3, 2'
  result['Conv_layer'] = 3
  result['MaxPooling_layer'] = 2
  result['accuracy_tr']= hh.history['accuracy'][-1]
  result['accuracy_val']= hh.history['val_accuracy'][-1]

  results.append(result)

2022-01-16 19:28:37.829832: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/2
469/469 [==============================] - 26s 54ms/step - loss: 1.0408 - accuracy: 0.6265 - val_loss: 0.7571 - val_accuracy: 0.7071
Epoch 2/2
469/469 [==============================] - 26s 56ms/step - loss: 0.6615 - accuracy: 0.7530 - val_loss: 0.6303 - val_accuracy: 0.7665
Epoch 1/2
469/469 [==============================] - 27s 58ms/step - loss: 1.0584 - accuracy: 0.6236 - val_loss: 0.7755 - val_accuracy: 0.7097
Epoch 2/2
469/469 [==============================] - 27s 58ms/step - loss: 0.6859 - accuracy: 0.7422 - val_loss: 0.6723 - val_accuracy: 0.7499
Epoch 1/2
469/469 [==============================] - 29s 61ms/step - loss: 1.0969 - accuracy: 0.6253 - val_loss: 0.7139 - val_accuracy: 0.7348
Epoch 2/2
469/469 [==============================] - 28s 60ms/step - loss: 0.6389 - accuracy: 0.7638 - val_loss: 0.5983 - val_accuracy: 0.7759
Epoch 1/2
469/469 [==============================] - 29s 61ms/step - loss: 1.0336 - accuracy: 0.6335 - val_loss: 0.7374 - val_accuracy: 0.7293

In [18]:
for i in range (11):
  result = {}

  # инициализация пустой модели
  model = Sequential()

  # первый сверточный слой
  model.add(layers.Conv2D(12, kernel_size=(10, 10), strides=(1, 1), activation='relu', input_shape=(28,28,1), padding="same"))

  # второй пуллинговый слой
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))

  # третий сверточный слой
  model.add(layers.Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='valid'))

  # четвертый пуллинговый слой
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

  # пятый сверточный слой
  model.add(layers.Conv2D(120, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='valid'))

  # сглаживание CNN выхода чтобы можно было его присоединить к полносвязногому слою
  model.add(layers.Flatten())

  # шестой полносвязный слой
  model.add(layers.Dense(84, activation='relu'))

  # выходной слой с функцией активации softmax
  model.add(layers.Dense(10, activation='softmax'))


  # компилияция модели
  model.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])


  hh = model.fit(x=x_train,y=y_train, epochs=2, batch_size=128, validation_data=(x_test, y_test), verbose=1)

  result['neurons'] = '12, 32, 120'
  result['kernels'] = '10, 5, 3'
  result['Conv_layer'] = 3
  result['MaxPooling_layer'] = 2
  result['accuracy_tr']= hh.history['accuracy'][-1]
  result['accuracy_val']= hh.history['val_accuracy'][-1]

  results.append(result)

Epoch 1/2
469/469 [==============================] - 42s 88ms/step - loss: 1.0823 - accuracy: 0.6028 - val_loss: 0.7162 - val_accuracy: 0.7304
Epoch 2/2
469/469 [==============================] - 41s 87ms/step - loss: 0.6396 - accuracy: 0.7623 - val_loss: 0.6141 - val_accuracy: 0.7639
Epoch 1/2
469/469 [==============================] - 42s 88ms/step - loss: 1.1369 - accuracy: 0.5972 - val_loss: 0.7765 - val_accuracy: 0.7105
Epoch 2/2
469/469 [==============================] - 41s 88ms/step - loss: 0.6178 - accuracy: 0.7692 - val_loss: 0.6381 - val_accuracy: 0.7655
Epoch 1/2
469/469 [==============================] - 41s 88ms/step - loss: 1.0635 - accuracy: 0.6255 - val_loss: 0.7074 - val_accuracy: 0.7385
Epoch 2/2
469/469 [==============================] - 41s 88ms/step - loss: 0.6390 - accuracy: 0.7637 - val_loss: 0.5963 - val_accuracy: 0.7801
Epoch 1/2
469/469 [==============================] - 41s 87ms/step - loss: 1.0666 - accuracy: 0.6201 - val_loss: 0.7173 - val_accuracy: 0.7244

In [19]:
for i in range (11):
  result = {}

  # инициализация пустой модели
  model = Sequential()

  # первый сверточный слой
  model.add(layers.Conv2D(12, kernel_size=(20, 20), strides=(1, 1), activation='relu', input_shape=(28,28,1), padding="same"))

  # второй пуллинговый слой
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))

  # третий сверточный слой
  model.add(layers.Conv2D(32, kernel_size=(10, 10), strides=(1, 1), activation='relu', padding='valid'))

  # четвертый пуллинговый слой
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

  # пятый сверточный слой
  model.add(layers.Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='valid'))

  # сглаживание CNN выхода чтобы можно было его присоединить к полносвязногому слою
  model.add(layers.Flatten())

  # шестой полносвязный слой
  model.add(layers.Dense(84, activation='relu'))

  # выходной слой с функцией активации softmax
  model.add(layers.Dense(10, activation='softmax'))


  # компилияция модели
  model.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])


  hh = model.fit(x=x_train,y=y_train, epochs=2, batch_size=128, validation_data=(x_test, y_test), verbose=1)

  result['neurons'] = '12, 32, 120'
  result['kernels'] = '20, 10, 5'
  result['Conv_layer'] = 3
  result['MaxPooling_layer'] = 2
  result['accuracy_tr']= hh.history['accuracy'][-1]
  result['accuracy_val']= hh.history['val_accuracy'][-1]

  results.append(result)

Epoch 1/2
469/469 [==============================] - 103s 218ms/step - loss: 1.0911 - accuracy: 0.6039 - val_loss: 0.7495 - val_accuracy: 0.7097
Epoch 2/2
469/469 [==============================] - 106s 226ms/step - loss: 0.6364 - accuracy: 0.7616 - val_loss: 0.5840 - val_accuracy: 0.7829
Epoch 1/2
469/469 [==============================] - 106s 225ms/step - loss: 1.0080 - accuracy: 0.6321 - val_loss: 0.6710 - val_accuracy: 0.7464
Epoch 2/2
469/469 [==============================] - 110s 234ms/step - loss: 0.5966 - accuracy: 0.7780 - val_loss: 0.5652 - val_accuracy: 0.7959
Epoch 1/2
469/469 [==============================] - 102s 216ms/step - loss: 1.0053 - accuracy: 0.6325 - val_loss: 0.7147 - val_accuracy: 0.7419
Epoch 2/2
469/469 [==============================] - 101s 216ms/step - loss: 0.6001 - accuracy: 0.7775 - val_loss: 0.6681 - val_accuracy: 0.7390
Epoch 1/2
469/469 [==============================] - 104s 222ms/step - loss: 0.9999 - accuracy: 0.6424 - val_loss: 0.6924 - val_ac

#### Оценка роста точности при увеличении глубины сети (больше слоев)

In [20]:
#2с 1п, 4с 3п, 5с 4п
for i in range (11):
  result = {}

  model = Sequential()

  model.add(layers.Conv2D(6, kernel_size=(6, 6), strides=(1, 1), activation='relu', padding="same"))

  model.add(layers.Conv2D(12, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=(28,28,1), padding='valid'))

  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))

  model.add(layers.Conv2D(32, kernel_size=(4, 4), strides=(1, 1), activation='relu', padding='valid'))

  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

  model.add(layers.Conv2D(80, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='valid'))

  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

  model.add(layers.Conv2D(80, kernel_size=(2, 2), strides=(1, 1), activation='relu', padding='valid'))

  
  # сглаживание CNN выхода чтобы можно было его присоединить к полносвязногому слою
  model.add(layers.Flatten())

  # шестой полносвязный слой
  model.add(layers.Dense(84, activation='relu'))

  # выходной слой с функцией активации softmax
  model.add(layers.Dense(10, activation='softmax'))

  # компилияция модели
  model.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])


  hh = model.fit(x=x_train,y=y_train, epochs=2, batch_size=128, validation_data=(x_test, y_test), verbose=1)

  result['neurons'] = '6, 12, 32, 80, 80'
  result['kernels'] = '6, 5, 4, 3, 2'
  result['Conv_layer'] = 5
  result['MaxPooling_layer'] = 3
  result['accuracy_tr']= hh.history['accuracy'][-1]
  result['accuracy_val']= hh.history['val_accuracy'][-1]

  results.append(result)

Epoch 1/2
469/469 [==============================] - 34s 72ms/step - loss: 1.4931 - accuracy: 0.4525 - val_loss: 0.8627 - val_accuracy: 0.6829
Epoch 2/2
469/469 [==============================] - 33s 71ms/step - loss: 0.7340 - accuracy: 0.7258 - val_loss: 0.7974 - val_accuracy: 0.7055
Epoch 1/2
469/469 [==============================] - 34s 71ms/step - loss: 1.6379 - accuracy: 0.4415 - val_loss: 0.7947 - val_accuracy: 0.7053
Epoch 2/2
469/469 [==============================] - 34s 72ms/step - loss: 0.6850 - accuracy: 0.7471 - val_loss: 0.6344 - val_accuracy: 0.7694
Epoch 1/2
469/469 [==============================] - 34s 72ms/step - loss: 1.7034 - accuracy: 0.3842 - val_loss: 0.8414 - val_accuracy: 0.6832
Epoch 2/2
469/469 [==============================] - 34s 73ms/step - loss: 0.7137 - accuracy: 0.7318 - val_loss: 0.6743 - val_accuracy: 0.7316
Epoch 1/2
469/469 [==============================] - 35s 73ms/step - loss: 1.6307 - accuracy: 0.4114 - val_loss: 0.8902 - val_accuracy: 0.6687

#### Сравнение точности полносвязной сети для этой выборки

In [21]:
(train_imagesi,train_labels),(test_imagesi,test_labels)=fashion_mnist.load_data()

In [22]:
# Normalize the images.
train_images = (train_imagesi / 127) - 1
test_images = (test_imagesi / 127) - 1

# Flatten the images.
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

In [23]:
for i in range (11):
  result = {}

  model = Sequential()

  model.add(layers.Dense(90, activation='relu', input_shape=(784,)))

  model.add(layers.Dense(90, activation='relu'))

  # выходной слой с функцией активации softmax
  model.add(layers.Dense(10, activation='softmax'))

  # компилияция модели
  model.compile(loss=keras.losses.categorical_crossentropy, optimizer='Nadam', metrics=["accuracy"])


  hh = model.fit(x=train_images,y=to_categorical(train_labels), epochs=2, batch_size=128, validation_data=(test_images, to_categorical(test_labels)), verbose=1)

  result['neurons'] = ''
  result['kernels'] = ''
  result['Conv_layer'] = 0
  result['MaxPooling_layer'] = 0
  result['accuracy_tr']= hh.history['accuracy'][-1]
  result['accuracy_val']= hh.history['val_accuracy'][-1]

  results.append(result)

Epoch 1/2
469/469 [==============================] - 2s 3ms/step - loss: 0.5184 - accuracy: 0.8154 - val_loss: 0.4305 - val_accuracy: 0.8435
Epoch 2/2
469/469 [==============================] - 1s 3ms/step - loss: 0.3740 - accuracy: 0.8640 - val_loss: 0.3878 - val_accuracy: 0.8600
Epoch 1/2
469/469 [==============================] - 2s 3ms/step - loss: 0.5170 - accuracy: 0.8148 - val_loss: 0.4312 - val_accuracy: 0.8451
Epoch 2/2
469/469 [==============================] - 1s 3ms/step - loss: 0.3674 - accuracy: 0.8652 - val_loss: 0.4141 - val_accuracy: 0.8516
Epoch 1/2
469/469 [==============================] - 2s 3ms/step - loss: 0.5128 - accuracy: 0.8154 - val_loss: 0.4331 - val_accuracy: 0.8452
Epoch 2/2
469/469 [==============================] - 1s 2ms/step - loss: 0.3684 - accuracy: 0.8658 - val_loss: 0.3936 - val_accuracy: 0.8580
Epoch 1/2
469/469 [==============================] - 2s 3ms/step - loss: 0.5187 - accuracy: 0.8145 - val_loss: 0.4816 - val_accuracy: 0.8231
Epoch 2/2
469

In [24]:
result1 = pd.DataFrame(results)

In [25]:
result1.groupby(['neurons', 'kernels','Conv_layer', 'MaxPooling_layer']).mean(['accuracy_val', 'accuracy_tr']).sort_values(by='accuracy_val', ascending=False)

accuracy_tr  \
neurons           kernels       Conv_layer MaxPooling_layer                
                                0          0                    0.864447   
12, 32, 120       10, 5, 3      3          2                    0.766226   
                  20, 10, 5     3          2                    0.776417   
                  5, 3, 2       3          2                    0.755308   
6, 12, 32, 80, 80 6, 5, 4, 3, 2 5          3                    0.734829   

                                                             accuracy_val  
neurons           kernels       Conv_layer MaxPooling_layer                
                                0          0                     0.857482  
12, 32, 120       10, 5, 3      3          2                     0.776236  
                  20, 10, 5     3          2                     0.774255  
                  5, 3, 2       3          2                     0.759009  
6, 12, 32, 80, 80 6, 5, 4, 3, 2 5          3                     0.744800

**Вывод:**

Результат тем лучше, чем меньше сверточных и пуллинг слоев.

#### Вариант 2. (сложный)

- реализовать нейронную сеть или стек из сверток (Concatenate) на сifar10.

- оценить рост точности при увеличении ширины сети (больше ядер), больше нитей.

- оценить рост точности при увеличении глубины сети (больше слоев)